<a href="https://colab.research.google.com/github/JakobUniver/Traffic-sign-detection-in-color-videos/blob/main/TestYOLO_LENET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/JakobUniver/Traffic-sign-detection-in-color-videos.git -q

In [2]:
!git clone https://github.com/WongKinYiu/yolov7.git -q

In [3]:
# Yolo
import numpy as np
import pandas as pd
import cv2
import torch
from google.colab.patches import cv2_imshow
# Lenet Imports
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy
import tensorflow as tf

from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Conv2D, Activation, Flatten, Dense, MaxPooling2D, BatchNormalization, Dropout

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#YOLO part

In [5]:
Yolo_model = torch.hub.load('yolov7', 'custom', 'Traffic-sign-detection-in-color-videos/model.pt', source='local') 

requirements: jedi>=0.10 not found and is required by YOLOR, attempting auto-update...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

requirements: 1 package updated per /content/yolov7/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

Adding autoShape... 


# Lenet part

In [6]:
crop_shape=(64,64)

In [7]:
lenet_model = keras.models.load_model('Traffic-sign-detection-in-color-videos/lenet_model')

In [8]:
classes = pd.read_csv('Traffic-sign-detection-in-color-videos/lenet_classes.csv')
classes = classes.columns[1:]

' Turn right ahead'

# Combined model

In [11]:
#cap = cv2.VideoCapture('traffic-sign-to-test.mp4')
cap = cv2.VideoCapture('drive/MyDrive/DashCamTestShort.mp4')

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
   
size = (frame_width, frame_height)
result = cv2.VideoWriter('resultDashCamTest.mp4', cv2.VideoWriter_fourcc(*'MP4V'), 30, size)
 
if (cap.isOpened()== False): 
  print("Error opening video stream or file")
 
while(cap.isOpened()):
  ret, frame = cap.read()
  if ret == True:
    
    detected = Yolo_model(frame)
    new_frame = frame.copy()
    for i, d in detected.pandas().xyxy[0].iterrows():

      crop = frame[int(d.ymin):int(np.ceil(d.ymax)), int(d.xmin):int(np.ceil(d.xmax))]
      new_crop = cv2.resize(crop, crop_shape, interpolation=cv2.INTER_AREA)
      pred = lenet_model.predict(np.array([new_crop]), verbose=0).argmax()
      cv2.rectangle(new_frame, (int(d.xmin), int(d.ymin)), (int(np.ceil(d.xmax)), int(np.ceil(d.ymax))), (255,0,0), 2)
      new_frame = cv2.putText(new_frame, classes[pred], (int(d.xmin), int(d.ymin)),
                              cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1, cv2.LINE_AA)
      
      """
      if len(crop) > 0:
        cv2_imshow(crop)
      """
      

    result.write(new_frame)
    cv2.waitKey(1) & 0xff
 
  else: 
    break

cv2.destroyAllWindows()
cap.release()
result.release()